In this assignment, you will be implementing a GPT model and train it using CLM objective.
 * If you get stuck at something or need more clarrifications, you may refer to : https://github.com/karpathy/minGPT/blob/master/mingpt/model.py

 * As usual, let us install the required libraries

 * **Note** that if you are not getting the exact loss values as mentioned in this notebook, that is absolutely fine. Just see whether your implementation overfits the given toy-and-tiny paragraph!

# Installation


In [2]:
!pip install torchdata==0.6.0 # to be compatible with torch 2.0
!pip install portalocker==2.0.0

* See [here](https://github.com/pytorch/text) for compatability

In [3]:
!pip install -U torchtext==0.15.1

# Imports

In [ ]:
import torch
from torch import Tensor

import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.functional import one_hot

import torch.optim as optim

#text lib
import torchtext

# tokenizer
from torchtext.data.utils import get_tokenizer

#build vocabulary
from torchtext.vocab import vocab
from torchtext.vocab import build_vocab_from_iterator

# get input_ids (numericalization)
from torchtext.transforms import VocabTransform

# get embeddings
from torch.nn import Embedding

from  pprint import pprint
from yaml import safe_load
import copy
import numpy as np

In [1]:
import nltk
nltk.download('punkt')

# Load the dataset for LM modeling

 * We use a simple tokenizer and put

In [ ]:
batch_size = 10

In [ ]:
class Tokenizer(object):

  def __init__(self,text):
    self.text = text
    self.word_tokenizer = get_tokenizer(tokenizer="basic_english",language='en')
    self.vocab_size = None

  def get_tokens(self):
    for sentence in self.text.strip().split('\n'):
      yield self.word_tokenizer(sentence)

  def build_vocab(self):
    v = build_vocab_from_iterator(self.get_tokens(),
                                  min_freq=1,specials=['<unk>','<start>','<end>'])
    v.set_default_index(v['<unk>']) # index of OOV
    self.vocab_size = len(v)
    return v

  def token_ids(self):
    v = self.build_vocab()
    vt = VocabTransform(v)
    num_tokens = len(self.word_tokenizer(self.text))
    max_seq_len = np.ceil(num_tokens/batch_size)
    data = torch.zeros(size=(1,num_tokens))
    data = vt(self.word_tokenizer(self.text))
    data = torch.tensor(data,dtype=torch.int64)
    return data.reshape(batch_size,torch.tensor(max_seq_len,dtype=torch.int64))



In [ ]:
text = """Best known for the invention of Error Correcting Codes, he was a true polymath who applied his mathematical and problem-solving skills to numerous disciplines.
Reflecting on the significant benefits I received from Hamming, I decided to develop a tribute to his legacy. There has not been a previous biography of Hamming, and the few articles about him restate known facts and assumptions and leave us with open questions.
One thought drove me as I developed this legacy project: An individual's legacy is more than a list of their attempts and accomplishments. Their tribute should also reveal the succeeding generations they inspired and enabled and what each attempted and achieved.
This book is a unique genre containing my version of a biography that intertwines the story "of a life" and a multi-player memoir with particular events and turning points recalled by those, including me, who he inspired and enabled.
Five years of research uncovered the people, places, opportunities, events, and influences that shaped Hamming. I discovered unpublished information, stories, photographs, videos, and personal remembrances to chronicle his life, which helped me put Hamming's
legacy in the context I wanted.The result demonstrates many exceptional qualities, including his noble pursuit of excellence and helping others. Hamming paid attention to the details, his writings continue to influence, and his guidance is a timeless gift to the world.
This biography is part of """

In [ ]:
Tk = Tokenizer(text)

In [ ]:
x_raw = Tk.token_ids()
print(x_raw.shape)

torch.Size([10, 26])


In [ ]:
# let us display the first 10 tokens of the vocabulary
v = Tk.build_vocab()
pprint(v.vocab.get_itos()[0:10])

['<unk>', '<start>', '<end>', ',', 'and', '.', 'the', 'a', 'of', 'to']


* Create the input_ids and Labels from the raw input sequence

In [ ]:
bs,raw_seq_len = x_raw.shape
x = torch.empty(size=(bs,raw_seq_len+2),dtype=torch.int64)
x[:,1:-1] =x_raw

# insert the index of special tokens
x[:,0] = torch.full(size=(1,batch_size),fill_value=v.vocab.get_stoi()['<start>'])
x[:,-1] = torch.full(size=(1,batch_size),fill_value=v.vocab.get_stoi()['<end>'])

#Quickly check implem
v = Tk.build_vocab()
words = []
for idx in x[0,:]:
  words.append(v.vocab.get_itos()[idx.item()])
print(' '.join(words))

<start> best known for the invention of error correcting codes , he was a true polymath who applied his mathematical and problem-solving skills to numerous disciplines . <end>


In [ ]:
# labels are just the input_ids shifted by right
bs,seq_len = x.shape
y = torch.empty(size=(bs,seq_len),dtype=torch.int64)
y[:,0:-1] = copy.deepcopy(x[:,1:])

#ignore the index of padded tokens while computing loss
y[:,-1] = torch.full(size=(1,batch_size),fill_value=-100)

# Configuration

In [ ]:
vocab_size = Tk.vocab_size
seq_len = x.shape[1]
embed_dim = 32
dmodel = embed_dim
dq = torch.tensor(4)
dk = torch.tensor(4)
dv = torch.tensor(4)
heads = torch.tensor(8)
d_ff = 4*dmodel

* Define all the sub-layers (mhma,ffn) in the transformer blocks
* Seed for $W_Q,W_K,W_V,W_O$, 43, 44 and 45, 46, respectively
* Seed for ffn $W_1,W_2$,  47 and 48. There are no biases
* Seed for output layer 49

In [ ]:
class MHMA(nn.Module):
  pass

class FFN(nn.Module):
  pass

class PredictionHead(nn.Module):
  pass


class PositionalEncoding(nn.Module):
  pass



In [ ]:
class DecoderLayer(nn.Module):

  def __init__(self,dmodel,dq,dk,dv,d_ff,heads,mask=None):
    super(DecoderLayer,self).__init__()
    self.mhma = MHMA(dmodel,dq,dk,dv,heads,mask=None)
    self.layer_norm_1 = torch.nn.LayerNorm(dmodel)
    self.layer_norm_2 = torch.nn.LayerNorm(dmodel)
    self.ffn = FFN(dmodel,d_ff)

  def forward(self,dec_rep):
    """
    your code goes here
    """
    return out

In [ ]:
class Embed(nn.Module):

  def __init__(self,vocab_size,embed_dim):
    super(Embed,self).__init__()
    self.embed = None # seed 70
    self.pe = None

  def forward(self,x):
    out = self.pe(self.embed(x))
    return out

In [ ]:
class Decoder(nn.Module):

  def __init__(self,vocab_size,dmodel,dq,dk,dv,d_ff,heads,mask,num_layers=1):
    super(Decoder,self).__init__()
    self.embed_lookup = Embed(vocab_size,embed_dim)
    self.dec_layers = nn.ModuleList(copy.deepcopy(DecoderLayer(dmodel,dq,dk,dv,d_ff,heads,mask)) for i in range(num_layers))
    self.predict = PredictionHead(dmodel,vocab_size)

  def forward(self,input_ids):
    out = self.embed_lookup(input_ids)
    for dec_layer in self.dec_layers:
      out = dec_layer(out)
    out = self.predict(out)

    return out

In [ ]:
model = Decoder(vocab_size,dmodel,dq,dk,dv,d_ff,heads,mask=None)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def train(input_ids,labels,epochs=1000):
  #loss_trace = []
  for epoch in range(epochs):
    out = model(input_ids)
    loss = None
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

In [4]:
# run the model for 10K epochs
train(x,y,10000)

The loss is about 0.09 after 10K epochs

# Generate text

In [ ]:
@torch.inference_mode()
def generate(model,prompt='<start>',max_words=10):



  return

In [ ]:
generate(model,prompt='<start>',max_words=25)

'biography that intertwines the story of a life and a multi-player memoir with particular events and turning points recalled by those , including me ,'

* Note the model has memorized the sentence from the training set. Given the start token, if your implementation reproduce a sentence as is in the training set, then your implementation is likely to be correct.
* Suppose the prompt is `<start> best known`, then we expect the model to produce the first sentence as is

In [ ]:
generate(model,prompt=['<start>','best','known'],max_words=25)

'for the invention of error correcting codes , he was a true polymath who applied his mathematical and problem-solving skills to numerous disciplines'

* Change the prompt

In [ ]:
generate(model,prompt=['<start>','reflecting','on'],max_words=25)

'the significant benefits i received from hamming , i decided to develop a tribute to his legacy . there has not been a'